# TFX Components Walk-through

## Learning Objectives

1.  Develop a high level understanding of TFX pipeline components.
2.  Learn how to use a TFX Interactive Context for prototype development of TFX pipelines.
3.  Work with the Tensorflow Data Validation (TFDV) library to check and analyze input data.
4.  Utilize the Tensorflow Transform (TFT) library for scalable data preprocessing and feature transformations.
5.  Employ the Tensorflow Model Analysis (TFMA) library for model evaluation.

In this lab, you will work with the [Covertype Data Set](https://github.com/jarokaz/mlops-labs/blob/master/datasets/covertype/README.md) and use TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features.

You will utilize  **TFX Interactive Context** to work with the TFX components interactivelly in a Jupyter notebook environment. Working in an interactive notebook is useful when doing initial data exploration, experimenting with models, and designing ML pipelines. You should be aware that there are differences in the way interactive notebooks are orchestrated, and how they access metadata artifacts. In a production deployment of TFX on GCP, you will use an orchestrator such as Kubeflow Pipelines, or Cloud Composer. In an interactive mode, the notebook itself is the orchestrator, running each TFX component as you execute the notebook cells. In a production deployment, ML Metadata will be managed in a scalabe database like MySQL, and artifacts in apersistent store such as Google Cloud Storage. In an interactive mode, both properties and payloads are stored in a local file system of the Jupyter host.

**Setup Note:**
Currently, TFMA visualizations do not render properly in JupyterLab. It is recommended to run this notebook in Jupyter Classic Notebook. To switch to Classic Notebook select *Launch Classic Notebook* from the *Help* menu.

In [42]:
!pip install tensorflow_data_validation
!pip install tensorflow_model_analysis
!pip install tfx

In [43]:
import os
import time
from pprint import pprint

import absl
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.components import (
    CsvExampleGen,
    Evaluator,
    ExampleValidator,
    InfraValidator,
    Pusher,
    SchemaGen,
    StatisticsGen,
    Trainer,
    Transform,
)
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.components.base import executor_spec
from tfx.dsl.components.common.importer import Importer
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy,
)
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.experimental.interactive.interactive_context import (
    InteractiveContext,
)
from tfx.proto import (
    example_gen_pb2,
    infra_validator_pb2,
    pusher_pb2,
    trainer_pb2,
)
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

**Note**: this lab was developed and tested with the following TF ecosystem package versions:

`Tensorflow Version: 2.6.2`  
`TFX Version: 1.4.0`  
`TFDV Version: 1.4.0`  
`TFMA Version:  0.35.0`

If you encounter errors with the above imports (e.g. TFX component not found), check your package versions in the cell below.

In [44]:
print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)
print("TFDV Version:", tfdv.__version__)
print("TFMA Version:", tfma.VERSION_STRING)

absl.logging.set_verbosity(absl.logging.INFO)

Tensorflow Version: 2.8.0
TFX Version: 1.7.0
TFDV Version: 1.7.0
TFMA Version: 0.38.0


If the versions above do not match, update your packages in the current Jupyter kernel below. The default `%pip` package installation location is not on your system installation PATH; use the command below to append the local installation path to pick up the latest package versions. Note that you may also need to restart your notebook kernel to pick up the specified package versions and re-run the imports cell above before proceeding with the lab.

In [45]:
os.environ["PATH"] += os.pathsep + "/home/jupyter/.local/bin"

## Configure lab settings

Set constants, location paths and other environment settings. 

In [46]:
ARTIFACT_STORE = os.path.join(os.sep, "home", "jupyter", "artifact-store")
SERVING_MODEL_DIR = os.path.join(os.sep, "home", "jupyter", "serving_model")
DATA_ROOT = "../../data"

## Creating Interactive Context

TFX Interactive Context allows you to create and run TFX Components in an interactive mode. It is designed to support experimentation and development in a Jupyter Notebook environment. It is an experimental feature and major changes to interface and functionality are expected. When creating the interactive context you can specifiy the following parameters:
- `pipeline_name` - Optional name of the pipeline for ML Metadata tracking purposes. If not specified, a name will be generated for you.
- `pipeline_root` - Optional path to the root of the pipeline's outputs. If not specified, an ephemeral temporary directory will be created and used.
- `metadata_connection_config` - Optional `metadata_store_pb2.ConnectionConfig` instance used to configure connection to a ML Metadata connection. If not specified, an ephemeral SQLite MLMD connection contained in the pipeline_root directory with file name "metadata.sqlite" will be used.


In [47]:
PIPELINE_NAME = "tfx-covertype-classifier"
PIPELINE_ROOT = os.path.join(
    ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S")
)

In [48]:
os.makedirs(PIPELINE_ROOT, exist_ok=True)

In [49]:
context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None,
)

## Ingesting data using ExampleGen

In any ML development process the first step  is to ingest the training and test datasets. The `ExampleGen` component ingests data into a TFX pipeline. It consumes external files/services to generate a set file files in the `TFRecord` format,  which will be used by other TFX components. It can also shuffle the data and split into an arbitrary number of partitions.

<img src=../../images/ExampleGen.png width="300">

### Configure and run CsvExampleGen

In this exercise, you use the `CsvExampleGen` specialization of `ExampleGen` to ingest CSV files from a GCS location and emit them as `tf.Example` records for consumption by downstream TFX pipeline components. Your task is to configure the component to create 80-20 `train` and `eval` splits. *Hint: review the [ExampleGen proto](https://github.com/tensorflow/tfx/blob/master/tfx/proto/example_gen.proto#L78) definition to split your data with hash buckets.*

In [50]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
            example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
        ]
    )
)

example_gen = tfx.components.CsvExampleGen(
    input_base=DATA_ROOT, output_config=output_config
).with_id("CsvExampleGen")

In [51]:
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data ../../data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Examine the ingested data

In [52]:
examples_uri = example_gen.outputs["examples"].get()[-1].uri
tfrecord_filenames = [
    os.path.join(examples_uri, "Split-train", name)
    for name in os.listdir(os.path.join(examples_uri, "Split-train"))
]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    for name, feature in example.features.feature.items():
        if feature.HasField("bytes_list"):
            value = feature.bytes_list.value
        if feature.HasField("float_list"):
            value = feature.float_list.value
        if feature.HasField("int64_list"):
            value = feature.int64_list.value
        print(f"{name}: {value}")
    print("******")

Cover_Type: [1]
Hillshade_Noon: [247]
Slope: [9]
Hillshade_3pm: [157]
Aspect: [183]
Vertical_Distance_To_Hydrology: [101]
Wilderness_Area: [b'Commanche']
Soil_Type: [b'C7757']
Horizontal_Distance_To_Roadways: [757]
Elevation: [3142]
Hillshade_9am: [223]
Horizontal_Distance_To_Fire_Points: [1871]
Horizontal_Distance_To_Hydrology: [648]
******
Horizontal_Distance_To_Hydrology: [60]
Wilderness_Area: [b'Cache']
Aspect: [124]
Vertical_Distance_To_Hydrology: [9]
Slope: [16]
Cover_Type: [2]
Hillshade_Noon: [227]
Horizontal_Distance_To_Roadways: [124]
Elevation: [1967]
Hillshade_3pm: [105]
Hillshade_9am: [245]
Horizontal_Distance_To_Fire_Points: [451]
Soil_Type: [b'C2704']
******


## Generating statistics using StatisticsGen

The `StatisticsGen`  component generates data statistics that can be used by other TFX components. StatisticsGen uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started). `StatisticsGen` generate statistics for each split in the `ExampleGen` component's output. In our case there two splits: `train` and `eval`.

<img src=../../images/StatisticsGen.png width="200">

### Configure and  run the `StatisticsGen` component

In [53]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs["examples"]
).with_id("StatisticsGen")

INFO:absl:Excluding no splits because exclude_splits is not set.


In [54]:
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

### Visualize statistics

The generated statistics can be visualized using the `tfdv.visualize_statistics()` function from the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library or using a utility method of the `InteractiveContext` object. In fact, most of the artifacts generated by the TFX components can be visualized using `InteractiveContext`.

In [55]:
context.show(statistics_gen.outputs["statistics"])

## Infering data schema using SchemaGen

Some TFX components use a description input data called a schema. The schema is an instance of `schema.proto`. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. `SchemaGen` automatically generates the schema by inferring types, categories, and ranges from data statistics. The auto-generated schema is best-effort and only tries to infer basic properties of the data. It is expected that developers review and modify it as needed. `SchemaGen` uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

The `SchemaGen` component generates the schema using the statistics for the `train` split. The statistics for other splits are ignored.

<img src=../../images/SchemaGen.png width="200">

### Configure and run the `SchemaGen` components

In [56]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"], infer_feature_shape=False
).with_id("SchemaGen")

INFO:absl:Excluding no splits because exclude_splits is not set.


In [57]:
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

### Visualize the inferred schema

In [58]:
context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,single,'Soil_Type'
'Wilderness_Area',STRING,required,single,'Wilderness_Area'
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,-
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Updating the auto-generated schema

In most cases the auto-generated schemas must be fine-tuned manually using insights from data exploration and/or domain knowledge about the data. For example, you know that in the `covertype` dataset there are seven types of forest cover (coded using 1-7 range) and that the value of the `Slope` feature should be in the 0-90 range. You can manually add these constraints to the auto-generated schema by setting the feature domain.

### Load the auto-generated schema proto file

In [59]:
schema_proto_path = "{}/{}".format(
    schema_gen.outputs["schema"].get()[0].uri, "schema.pbtxt"
)
schema = tfdv.load_schema_text(schema_proto_path)

### Modify the schema

You can use the protocol buffer APIs to modify the schema. 

**Hint**: Review the [TFDV library API documentation](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/set_domain) on setting a feature's domain. You can use the protocol buffer APIs to modify the schema. Review the [Tensorflow Metadata proto definition](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto#L405) for configuration options.

In [60]:
# TODO: Your code to restrict the categorical feature Cover_Type between the values of 0 and 6.
# TODO: Your code to restrict the numeric feature Slope between 0 and 90.

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,single,'Soil_Type'
'Wilderness_Area',STRING,required,single,'Wilderness_Area'
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,-
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


#### Save the updated schema 

In [61]:
schema_dir = os.path.join(ARTIFACT_STORE, "schema")
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, "schema.pbtxt")

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "Soil_Type"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "Soil_Type"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Wilderness_Area"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "Wilderness_Area"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Aspect"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Cover_Type"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Elevation"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_3pm"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_9am"
  value_count {
    min: 1
    max: 1
  }
  type: IN

## Importing the updated schema using Importer

The `Importer` component allows you to import an external artifact, including the schema file, so it can be used by other TFX components in your workflow. 


### Configure and run the `Importer` component

In [62]:
schema_importer = Importer(
    source_uri=schema_dir, artifact_type=tfx.types.standard_artifacts.Schema
).with_id("SchemaImporter")

In [63]:
context.run(schema_importer)

INFO:absl:Running driver for SchemaImporter
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: /home/jupyter/artifact-store/schema, properties: {}, custom_properties: {}
INFO:absl:Running executor for SchemaImporter
INFO:absl:Running publisher for SchemaImporter
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaImporter
    execution_id: 4
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 18
        uri: "/home/jupyter/artifact-store/schema"
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.7.0"
          }
        }
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Visualize the imported schema

In [64]:
context.show(schema_importer.outputs["result"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,single,'Soil_Type'
'Wilderness_Area',STRING,required,single,'Wilderness_Area'
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,-
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Validating data with ExampleValidator

The `ExampleValidator` component identifies anomalies in data.  It identifies anomalies by comparing data statistics computed by the `StatisticsGen` component against a schema generated by `SchemaGen` or imported by `Importer`.

`ExampleValidator` can detect different classes of anomalies. For example it can:

- perform validity checks by comparing data statistics against a schema 
- detect training-serving skew by comparing training and serving data.
- detect data drift by looking at a series of data.


The `ExampleValidator` component validates the data in the `eval` split only. Other splits are ignored. 

<img src=../../images/ExampleValidator.png width="350">

### Configure and run the `ExampleValidator` component


In [65]:
# TODO: Complete ExampleValidator
# Hint: review the visual above and review the documentation on ExampleValidator's inputs and outputs:
# https://www.tensorflow.org/tfx/guide/exampleval
# Make sure you use the output of the schema_importer component created above.

example_validator = ExampleValidator(
 statistics=statistics_gen.outputs['statistics'],
 schema=schema_importer.outputs['result'],
).with_id("ExampleValidator")

INFO:absl:Excluding no splits because exclude_splits is not set.


In [66]:
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/ExampleValidator/anomalies/5/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/ExampleValidator/anomalies/5/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

### Visualize validation results

The file `anomalies.pbtxt` can be visualized using `context.show`.

In [67]:
context.show(example_validator.outputs["anomalies"])

In our case no anomalies were detected in the `eval` split.

For a detailed deep dive into data validation and schema generation refer to the `lab-31-tfdv-structured-data` lab.

## Preprocessing data with Transform

The `Transform` component performs data transformation and feature engineering. The `Transform` component consumes `tf.Examples` emitted from the `ExampleGen` component and emits the transformed feature data and the `SavedModel` graph that was used to process the data. The emitted `SavedModel`  can then be used by serving components to make sure that the same data pre-processing logic is applied at training and serving.

The `Transform` component requires more code than many other components because of the arbitrary complexity of the feature engineering that you may need for the data and/or model that you're working with. It requires code files to be available which define the processing needed.

<img src=../../images/Transform.png width="400">

### Define the pre-processing module

To configure `Trainsform`, you need to encapsulate your pre-processing code in the Python `preprocessing_fn` function and save it to a  python module that is then provided to the Transform component as an input. This module will be loaded by transform and the `preprocessing_fn` function will be called when the `Transform` component runs.

In most cases, your implementation of the `preprocessing_fn` makes extensive use of [TensorFlow Transform](https://www.tensorflow.org/tfx/guide/tft) for performing feature engineering on your dataset.

In [68]:
TRANSFORM_MODULE = "preprocessing.py"
!cat {TRANSFORM_MODULE}

# Copyright 2021 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Covertype preprocessing.
This file defines a template for TFX Transform component.
"""

import features
import tensorflow as tf
import tensorflow_transform as tft


def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
    Args:
      x: A `SparseTensor` of rank 2.  Its dense shape should have size 

### Configure and run the `Transform` component.

In [69]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_importer.outputs["result"],
    module_file=TRANSFORM_MODULE,
).with_id("Transform")

In [70]:
context.run(transform)

INFO:absl:Generating ephemeral wheel package for '/home/jupyter/asl-ml-immersion/notebooks/tfx_pipelines/walkthrough/labs/preprocessing.py' (including modules: ['model', 'features', 'preprocessing']).
INFO:absl:User module package has hash fingerprint version d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmpsqel66nh/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp4pt11ldu', '--dist-dir', '/tmp/tmpsveegjfe']
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
INFO:absl:Successfully built user code wheel distribution at '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Transform-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl'; target user module

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying model.py -> build/lib
copying features.py -> build/lib
copying preprocessing.py -> build/lib
installing to /tmp/tmp4pt11ldu
running install
running install_lib
copying build/lib/preprocessing.py -> /tmp/tmp4pt11ldu
copying build/lib/model.py -> /tmp/tmp4pt11ldu
copying build/lib/features.py -> /tmp/tmp4pt11ldu
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp4pt11ldu/tfx_user_code_Transform-0.0+d186e5de4

INFO:absl:Successfully installed '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Transform-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl'.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'preprocessing@/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Transform-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Transform-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/opt/conda/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmp2cgmhwd6', '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Transform-0.0

Processing /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Transform-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl


INFO:absl:Successfully installed '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Transform-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl'.
INFO:absl:Installing '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Transform-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/opt/conda/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmpdxvr19so', '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Transform-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl']


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

### Examine the `Transform` component's outputs

The Transform component has 2 outputs:

- `transform_graph` - contains the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
- `transformed_examples` - contains the preprocessed training and evaluation data.

Take a peek at the `transform_graph` artifact: it points to a directory containing 3 subdirectories:

In [71]:
os.listdir(transform.outputs["transform_graph"].get()[0].uri)

['transformed_metadata', 'metadata', 'transform_fn']

And the `transform.examples` artifact

In [72]:
os.listdir(transform.outputs["transformed_examples"].get()[0].uri)

['Split-eval', 'Split-train']

In [73]:
transform_uri = transform.outputs["transformed_examples"].get()[0].uri
tfrecord_filenames = [
    os.path.join(transform_uri, "Split-train", name)
    for name in os.listdir(os.path.join(transform_uri, "Split-train"))
]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    for name, feature in example.features.feature.items():
        if feature.HasField("bytes_list"):
            value = feature.bytes_list.value
        if feature.HasField("float_list"):
            value = feature.float_list.value
        if feature.HasField("int64_list"):
            value = feature.int64_list.value
        print(f"{name}: {value}")
    print("******")

Elevation_xf: [0.651531994342804]
Wilderness_Area_xf: [1]
Slope_xf: [-0.6797884106636047]
Hillshade_9am_xf: [0.4025527536869049]
Hillshade_Noon_xf: [1.1993097066879272]
Horizontal_Distance_To_Hydrology_xf: [1.7856396436691284]
Horizontal_Distance_To_Fire_Points_xf: [-0.08278580754995346]
Cover_Type_xf: [1]
Horizontal_Distance_To_Roadways_xf: [-1.0252416133880615]
Hillshade_3pm_xf: [0.38384684920310974]
Vertical_Distance_To_Hydrology_xf: [0.9501996040344238]
Aspect_xf: [0.24982306361198425]
Soil_Type_xf: [3]
******
Slope_xf: [0.25757989287376404]
Hillshade_3pm_xf: [-0.9800227284431458]
Hillshade_9am_xf: [1.2273873090744019]
Cover_Type_xf: [2]
Hillshade_Noon_xf: [0.1884450614452362]
Elevation_xf: [-3.5473971366882324]
Horizontal_Distance_To_Roadways_xf: [-1.4311398267745972]
Soil_Type_xf: [19]
Vertical_Distance_To_Hydrology_xf: [-0.6439015865325928]
Aspect_xf: [-0.2777790129184723]
Horizontal_Distance_To_Fire_Points_xf: [-1.1525152921676636]
Horizontal_Distance_To_Hydrology_xf: [-0.98328

## Train your TensorFlow model with the `Trainer` component

The `Trainer` component trains a model using TensorFlow.

`Trainer` takes:

- tf.Examples used for training and eval.
- A user provided module file that defines the trainer logic.
- A data schema created by `SchemaGen` or imported by `Importer`.
- A proto definition of train args and eval args.
- An optional transform graph produced by upstream Transform component.
- An optional base models used for scenarios such as warmstarting training.

<img src=../../images/Trainer.png width="400">


### Define the trainer module

To configure `Trainer`, you need to encapsulate your training code in a Python module that is then provided to the `Trainer` as an input. 


In [74]:
TRAINER_MODULE_FILE = "model.py"
!cat {TRAINER_MODULE_FILE}

# Copyright 2021 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Covertype Keras WideDeep Classifier."""

import functools
from typing import List

import absl
import features
import kerastuner
import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx.components.tuner.component import TunerFnResult
from tfx_bsl.tfxio import dataset_op

### Create and run the Trainer component

Note that the `Trainer` component supports passing the field `num_steps` through the `train_args` and `eval_args` arguments. 

In [75]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(
        trainer_executor.GenericExecutor
    ),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs["transformed_examples"],
    schema=schema_importer.outputs["result"],
    transform_graph=transform.outputs["transform_graph"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=2),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=1),
).with_id("Trainer")

In [76]:
context.run(trainer)

INFO:absl:Generating ephemeral wheel package for '/home/jupyter/asl-ml-immersion/notebooks/tfx_pipelines/walkthrough/labs/model.py' (including modules: ['model', 'features', 'preprocessing']).
INFO:absl:User module package has hash fingerprint version d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmpzguo76z3/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpcv9e2x2d', '--dist-dir', '/tmp/tmprqktua8g']
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
INFO:absl:Successfully built user code wheel distribution at '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Trainer-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl'; target user module is 'model

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying model.py -> build/lib
copying features.py -> build/lib
copying preprocessing.py -> build/lib
installing to /tmp/tmpcv9e2x2d
running install
running install_lib
copying build/lib/preprocessing.py -> /tmp/tmpcv9e2x2d
copying build/lib/model.py -> /tmp/tmpcv9e2x2d
copying build/lib/features.py -> /tmp/tmpcv9e2x2d
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpcv9e2x2d/tfx_user_code_Trainer-0.0+d186e5de41d97c5dededf4835ec

INFO:absl:Successfully installed '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/_wheels/tfx_user_code_Trainer-0.0+d186e5de41d97c5dededf4835eca2bcf2a39887badc1a02add802aee780e9b2f-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:Feature Aspect_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Cover_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Elevation_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_3pm_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_9am_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_Noon_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Slope_xf has a shape . Setting t

INFO:absl:Feature Aspect_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Cover_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Elevation_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_3pm_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_9am_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_Noon_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Slope_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Soil_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Vertical_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Wilderness_Area_xf has a shape . Setting to DenseTensor.

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/Trainer/model/7/Format-Serving/assets
INFO:absl:Training complete. Model written to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/Trainer/model/7/Format-Serving. ModelRun written to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/Trainer/model_run/7
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Analyzing training runs with TensorBoard

In this step you will analyze the training run with [TensorBoard.dev](https://blog.tensorflow.org/2019/12/introducing-tensorboarddev-new-way-to.html). `TensorBoard.dev` is a managed service that enables you to easily host, track and share your ML experiments.

#### Retrieve the location of TensorBoard logs

Each model run's train and eval metric logs are written to the `model_run` directory by the Tensorboard callback defined in `model.py`.

In [77]:
logs_path = trainer.outputs["model_run"].get()[0].uri
print(logs_path)

/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/Trainer/model_run/7


#### Upload the logs and start TensorBoard.dev

1. Open a new JupyterLab terminal window

2. From the terminal window, execute the following command
```
tensorboard dev upload --logdir [YOUR_LOGDIR]
```

Where [YOUR_LOGDIR] is an URI retrieved by the previous cell.

You will be asked to authorize `TensorBoard.dev` using your Google account. If you don't have a Google account or you don't want to authorize `TensorBoard.dev` you can skip this exercise.

After the authorization process completes, follow the link provided to view your experiment.

## Evaluating trained models with Evaluator
The `Evaluator` component analyzes model performance using the [TensorFlow Model Analysis library](https://www.tensorflow.org/tfx/model_analysis/get_started). It runs inference requests on particular subsets of the test dataset, based on which slices are defined by the developer. Knowing which slices should be analyzed requires domain knowledge of what is important in this particular use case or domain. 

The `Evaluator` can also optionally validate a newly trained model against a previous model. In this lab, you only train one model, so the Evaluator automatically will label the model as "blessed".


<img src=../../images/Evaluator.png width="400">

### Configure and run the Evaluator component


Use the `ResolverNode` to pick the previous model to compare against.  The model resolver is only required if performing model validation in addition to evaluation. In this case we validate against the latest blessed model. If no model has been blessed before (as in this case) the evaluator will make our candidate the first blessed model.

In [78]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=tfx.types.standard_artifacts.Model),
    model_blessing=Channel(type=tfx.types.standard_artifacts.ModelBlessing),
).with_id("LatestBlessedModelResolver")

In [79]:
context.run(model_resolver)

INFO:absl:Running driver for LatestBlessedModelResolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for LatestBlessedModelResolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: LatestBlessedModelResolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=LatestBlessedModelResolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=LatestBlessedModelResolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

Configure evaluation metrics and slices.

In [101]:
# TODO: Your code here to create a tfma.MetricThreshold.
# Review the API documentation here: https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold
# Hint: Review the API documentation for tfma.GenericValueThreshold to constrain accuracy between 50% and 99%.

accuracy_threshold = tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.0},upper_bound={'value':0.9}))

metrics_specs = tfma.MetricsSpec(
    metrics=[
        tfma.MetricConfig(
            class_name="SparseCategoricalAccuracy", threshold=accuracy_threshold
        ),
        tfma.MetricConfig(class_name="ExampleCount"),
    ]
)

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key="Cover_Type")],
    metrics_specs=[metrics_specs],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=["Wilderness_Area"]),
    ],
)
eval_config

model_specs {
  label_key: "Cover_Type"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "Wilderness_Area"
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
        }
        upper_bound {
          value: 0.9
        }
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

In [102]:
model_analyzer = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
).with_id("ModelEvaluator")

In [103]:
context.run(model_analyzer, enable_cache=False)

INFO:absl:Running driver for ModelEvaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ModelEvaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "SparseCategoricalAccuracy",\n          "threshold": {\n            "value_threshold": {\n              "lower_bound": 0.0,\n              "upper_bound": 0.9\n            }\n          }\n        },\n        {\n          "class_name": "ExampleCount"\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "Cover_Type"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "Wilderness_Area"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_eval_shared_model'
INFO:absl:Reques

INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "SparseCategoricalAccuracy",\n          "threshold": {\n            "value_threshold": {\n              "lower_bound": 0.0,\n              "upper_bound": 0.9\n            }\n          }\n        },\n        {\n          "class_name": "ExampleCount"\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "Cover_Type"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "Wilderness_Area"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_extractors'
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not pro

INFO:absl:Evaluation complete. Results written to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/ModelEvaluator/evaluation/12.
INFO:absl:Checking validation results.
INFO:absl:Blessing result True written to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/ModelEvaluator/blessing/12.
INFO:absl:Running publisher for ModelEvaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ModelEvaluator
    execution_id: 12
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=ModelEvaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=ModelEvaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

### Check the model performance validation status

In [104]:
model_blessing_uri = model_analyzer.outputs["blessing"].get()[0].uri
!ls -l {model_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Mar 31 17:02 BLESSED


### Visualize evaluation results
You can visualize the evaluation results using the `tfma.view.render_slicing_metrics()` function from TensorFlow Model Analysis library.

**Setup Note:** *Currently, TFMA visualizations don't render in  JupyterLab. Make sure that you run this notebook in Classic Notebook.*

In [105]:
evaluation_uri = model_analyzer.outputs["evaluation"].get()[0].uri
evaluation_uri
!ls {evaluation_uri}

attributions-00000-of-00001.tfrecord  plots-00000-of-00001.tfrecord
eval_config.json		      validations.tfrecord
metrics-00000-of-00001.tfrecord


In [106]:
eval_result = tfma.load_eval_result(evaluation_uri)
eval_result

EvalResult(slicing_metrics=[((('Wilderness_Area', 'Cache'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.07709414511919022}, 'loss': {'doubleValue': 2.1174304485321045}, 'example_count': {'doubleValue': 1349.0}}}}), ((), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.18244987726211548}, 'loss': {'doubleValue': 2.0324785709381104}, 'example_count': {'doubleValue': 20148.0}}}}), ((('Wilderness_Area', 'Commanche'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.1685214638710022}, 'loss': {'doubleValue': 2.042142868041992}, 'example_count': {'doubleValue': 8806.0}}}}), ((('Wilderness_Area', 'Rawah'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.22395890951156616}, 'loss': {'doubleValue': 1.990173101425171}, 'example_count': {'doubleValue': 8957.0}}}}), ((('Wilderness_Area', 'Neota'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.07915057986974716}, 'loss': {'doubleValue': 2.205474853515625}, 'example_count': {'do

In [107]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [108]:
tfma.view.render_slicing_metrics(eval_result, slicing_column="Wilderness_Area")

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Wilderness_Area:Cach…

## InfraValidator

The `InfraValidator` component acts as an additional early warning layer by validating a candidate model in a sandbox version of its serving infrastructure to prevent an unservable model from being pushed to production. Compared to the `Evaluator` component above which validates a model's performance, the `InfraValidator` component is validating that a model is able to generate predictions from served examples in an environment configured to match production. The config below takes a model and examples, launches the model in a sand-boxed [TensorflowServing](https://www.tensorflow.org/tfx/guide/serving) model server from the latest image in a local docker engine, and optionally checks that the model binary can be loaded and queried before "blessing" it for production.

<img src=../../images/InfraValidator.png width="400">

In [109]:
infra_validator = InfraValidator(
    model=trainer.outputs["model"],
    examples=example_gen.outputs["examples"],
    serving_spec=infra_validator_pb2.ServingSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(
            tags=["latest"]
        ),
        local_docker=infra_validator_pb2.LocalDockerConfig(),
    ),
    validation_spec=infra_validator_pb2.ValidationSpec(
        max_loading_time_seconds=60,
        num_tries=5,
    ),
    request_spec=infra_validator_pb2.RequestSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(),
        num_examples=5,
    ),
).with_id("ModelInfraValidator")

In [110]:
context.run(infra_validator, enable_cache=False)

INFO:absl:Running driver for ModelInfraValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ModelInfraValidator
INFO:absl:InfraValidator will be run in LOAD_AND_QUERY mode.
INFO:absl:tag_set is not given. Using {'serve'} instead.
INFO:absl:signature_names are not given. Using ['serving_default'] instead.
INFO:absl:Creating temp directory at /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/.temp/13/
INFO:absl:Starting infra validation (attempt 1/5).
INFO:absl:Starting LocalDockerRunner(image: tensorflow/serving:latest).
INFO:absl:Running container with parameter {'auto_remove': True, 'detach': True, 'publish_all_ports': True, 'image': 'tensorflow/serving:latest', 'environment': {'MODEL_NAME': 'infra-validation-model', 'MODEL_BASE_PATH': '/model'}, 'mounts': [{'Target': '/model/infra-validation-model/1', 'Source': '/home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/.temp/13/infra-validation-model/164874617

ExecutionResult(
    component_id: ModelInfraValidator
    execution_id: 13
    outputs:
        blessing: OutputChannel(artifact_type=InfraBlessing, producer_component_id=ModelInfraValidator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

### Check the model infrastructure validation status

In [111]:
infra_blessing_uri = infra_validator.outputs["blessing"].get()[0].uri
!ls -l {infra_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Mar 31 17:03 INFRA_BLESSED


## Deploying models with Pusher

The `Pusher` component checks whether a model has been "blessed", and if so, deploys it by pushing the model to a well known file destination.

<img src=../../images/Pusher.png width="400">



### Configure and run the `Pusher` component

In [112]:
trainer.outputs["model"]

OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})

In [113]:
pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=model_analyzer.outputs["blessing"],
    infra_blessing=infra_validator.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR
        )
    ),
).with_id("ModelPusher")

In [114]:
context.run(pusher)

INFO:absl:Running driver for ModelPusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ModelPusher
INFO:absl:Model version: 1648746191
INFO:absl:Model written to serving path /home/jupyter/serving_model/1648746191.
INFO:absl:Model pushed to /home/jupyter/artifact-store/tfx-covertype-classifier/20220331_164458/ModelPusher/pushed_model/14.
INFO:absl:Running publisher for ModelPusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ModelPusher
    execution_id: 14
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=ModelPusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

### Examine the output of `Pusher`

In [115]:
pusher.outputs

{'pushed_model': OutputChannel(artifact_type=PushedModel, producer_component_id=ModelPusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={})}

In [116]:
latest_pushed_model = os.path.join(
    SERVING_MODEL_DIR, max(os.listdir(SERVING_MODEL_DIR))
)

In [117]:
ls $latest_pushed_model

assets/  keras_metadata.pb  saved_model.pb  variables/


## Next steps

This concludes your introductory walthrough through TFX pipeline components. In the lab, you used TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features. You utilized a TFX Interactive Context for prototype development of a TFX pipeline directly in a Jupyter notebook. Next, you worked with the TFDV library to modify your dataset schema to add feature constraints to catch data anamolies that can negatively impact your model's performance. You utilized TFT library for feature proprocessing for consistent feature transformations for your model at training and serving time. Lastly, using the TFMA library, you added model performance constraints to ensure you only push more accurate models than previous runs to production.

## License

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.